In [1]:
# requirements = [
#     "langchain",
#     "langchain-community",
#     "llama-parse",
#     "fastembed",
#     "python-dotenv",
#     "langchain-groq",
#     "chainlit",
#     "sentence-transformers",
#     "openai",
#     "langchain-openai",
#     "nltk",
#     "joblib",
#     "gdown",
#     "PyPDF2",
#     "faiss-cpu",
#     "nest-asyncio",
#     "unstructured[md]"
# ]

# file_path = "requirements.txt"
# with open(file_path, "w") as f:
#     for package in requirements:
#         f.write(f"{package}\n")

In [ ]:
import os
from typing import Tuple, List, Dict
from pypdf import PdfReader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from openai import OpenAI

from PyPDF2 import PdfReader
from collections import deque
import json

In [3]:
# import gdown
# from PyPDF2 import PdfReader

# file_ids = [
#     '1ohQ7aQCiY4pKqkKl0_FssgYkkHmeYDWN',
#     '1SjDi9aY8_jQtDfe5wix-aFYuS5TP0pDG'
# ]

# for file_id in file_ids:
#     file_url = f'https://drive.googlea.com/uc?id={file_id}'
#     output_pdf = f'/content/{file_id}.pdf'
#     gdown.download(file_url, output_pdf, quiet=False)
#     reader = PdfReader(output_pdf)

#     for page_num in range(len(reader.pages)):
#         page = reader.pages[page_num]
#         print(f"Text from page {page_num + 1}:\n{page.extract_text()}\n")


In [ ]:
DATA_DIR = "./data"
PARSED_DATA_FILE = os.path.join(DATA_DIR, "parsed_data.pkl")
PDF_FILE = [
    'Data\\แผนปฏิบัติการการพัฒนาเมืองอุตสาหกรรมเชิงนิเวศพื้นที่ลาดหลุมแก้ว ระยะ 5 ปี (พศ-2566-2570).pdf',
    'Data\\รายงานการวิเคราะห์อุปสงค์และอุปทานจังหวัด.pdf'
]

def extract_text_from_pdf(pdf_file: str) -> str:
    reader = PdfReader(pdf_file)
    all_text = ""
    for page in reader.pages:
        all_text += page.extract_text()
    return all_text

def create_vector_database(
    llamaparse_api_key: str,
    pdf_files: list = PDF_FILE,
    data_file: str = PARSED_DATA_FILE,
) -> Tuple:
    os.makedirs(DATA_DIR, exist_ok=True)
    all_text = ""
    for pdf_file in pdf_files:
        all_text += extract_text_from_pdf(pdf_file)

    text_output = os.path.join(DATA_DIR, "extracted_text.txt")
    with open(text_output, "w", encoding="utf-8") as f:
        f.write(all_text)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=128)
    chunks = text_splitter.split_text(all_text)
    documents = [Document(page_content=chunk) for chunk in chunks]
    embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") #BAAI/bge-base-en-v1.5
    vector_store = FAISS.from_documents(documents=documents, embedding=embed_model)
    
    faiss_index_path = os.path.join(DATA_DIR, "faiss_index")
    vector_store.save_local(faiss_index_path)

    return vector_store, embed_model

In [5]:
client = OpenAI(
    api_key='sk-GqA4Uj6iZXaykbOzIlFGtmdJr6VqiX94NhhjPZaf81kylRzh',
    base_url='https://api.opentyphoon.ai/v1'
)

def summarize_text(text, max_tokens=5000):
    tokens = text.split()
    return ' '.join(tokens[:max_tokens]) + '...' if len(tokens) > max_tokens else text

class ConversationalMemory:
    def __init__(self, max_length=10):
        self.history = deque(maxlen=max_length)
    def add_to_memory(self, question: str, response: str | None):
        if response is not None :
            self.history.append({"question": question, "response": response})
    def get_memory(self) -> List[Dict[str, str]]:
        return list(self.history)
    def save_memory_to_file(self, file_path: str):
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(self.get_memory(), f, indent=4)
    def load_memory_from_file(self, file_path: str):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                self.history = deque(json.load(f), maxlen=self.history.maxlen)
        except FileNotFoundError:
            print(f"No existing memory file found at {file_path}. Starting fresh.")

def generate_response(prompt):
    chat_completion = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": prompt}]
    )
    return chat_completion.choices[0].message.content

def retrieve_documents(query, retriever):
    return retriever.get_relevant_documents(query)

def ask_question_with_memory(retriever, question, memory: ConversationalMemory):
    retrieved_docs = retrieve_documents(question, retriever)
    summarized_data = summarize_text("\n".join([doc.page_content for doc in retrieved_docs]), max_tokens=5000)
    history_context = "\n".join(
        [f"Q: {entry['question']}\nA: {entry['response']}" for entry in memory.get_memory()]
    )
    full_prompt = (
        f"Conversation history:\n{history_context}\n\n"
        f"Context for Pathum Thani development:\n{summarized_data}\n\n"
        f"New question: {question}"
    )
    response = generate_response(full_prompt)
    memory.add_to_memory(question, response)

    return response

llamaparse_api_key = "llx-pNes5rGZru1FvO1nINQMrAJMEso0OEWutgy8ejbGntSxNPeq"
vector_db, embed_model = create_vector_database(llamaparse_api_key)

retriever = vector_db.as_retriever(search_kwargs={'k': 3})

C:\Users\kongl\AppData\Local\Temp\ipykernel_21196\141400966.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") #BAAI/bge-base-en-v1.5
c:\Users\kongl\Documents\DSI314 Progress 2\DSI314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
if __name__ == "__main__":
    memory = ConversationalMemory(max_length=10)
    memory_file = "conversation_memory.json"
    memory.load_memory_from_file(memory_file)

    while True:
        question = input("Enter your question: ")
        if question.lower() == "exit":
            print("Goodbye!")
            break
        response = ask_question_with_memory(retriever, question, memory)
        print(f"Answer: {response}")
    memory.save_memory_to_file(memory_file)

Answer: # ข้อมูลเกี่ยวกับการพัฒนาด้าน OTOP

## OTOP ในจังหวัดปทุมธานี

OTOP หรือ One Tambon One Product เป็นโครงการที่ส่งเสริมการผลิตสินค้าและบริการของชุมชนในแต่ละตำบล เพื่อสร้างรายได้และสร้างความเข้มแข็งให้กับชุมชน

## สินค้า OTOP ที่โดดเด่น

1. ผ้าไหม: จังหวัดปทุมธานีมีชื่อเสียงในการผลิตผ้าไหมที่มีคุณภาพสูง
2. ผลิตภัณฑ์จากไม้: จังหวัดปทุมธานีมีทรัพยากรไม้ที่อุดมสมบูรณ์ ทำให้สามารถผลิตผลิตภัณฑ์จากไม้ได้หลากหลายชนิด
3. ผลิตภัณฑ์จากดอกไม้: จังหวัดปทุมธานีมีดอกไม้ที่สวยงามและหลากหลายชนิด ทำให้สามารถผลิตผลิตภัณฑ์จากดอกไม้ได้หลากหลายชนิด

## การส่งเสริม OTOP

1. การฝึกอบรม: รัฐบาลได้จัดโครงการฝึกอบรมให้กับผู้ประกอบการ OTOP เพื่อพัฒนาทักษะและความรู้ในการผลิตสินค้า
2. การสนับสนุนเงินทุน: รัฐบาลได้จัดโครงการสนับสนุนเงินทุนให้กับผู้ประกอบการ OTOP เพื่อช่วยเหลือในการลงทุนและขยายกิจการ
3. การประชาสัมพันธ์: รัฐบาลได้จัดโครงการประชาสัมพันธ์เพื่อสร้างความตระหนักและสร้างความสนใจในสินค้า OTOP

Reference(s):
[1] สำนักงานพัฒนาชุมชนจังหวัดปทุมธานี. (2564). รายงานผลการดำเนินงานโครงการ OTOP ปี 2564.
[2] ส

In [7]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True)

Fetching 30 files: 100%|██████████| 30/30 [00:00<?, ?it/s]


In [8]:
sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.", 
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]

embeddings_1 = model.encode(sentences_1, 
                            batch_size=12, 
                            max_length=8192,
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[[0.62590337 0.34749585]
 [0.34986818 0.6782464 ]]


In [9]:
%pip install llama-index-readers-faiss

Note: you may need to restart the kernel to use updated packages.


In [10]:
import faiss
import numpy as np

faiss_index_path = 'Data/faiss_index/index.faiss'
index = faiss.read_index(faiss_index_path)
num_vectors_to_retrieve = 10
vectors = []

for i in range(num_vectors_to_retrieve):
    vector = index.reconstruct(i)
    vectors.append(vector)

vectors = np.array(vectors)
print(f"Retrieved vectors: \n{vectors}")

query_vector = np.random.rand(index.d).astype(np.float32) 

k = 5

distances, indices = index.search(np.array([query_vector]), k)
print(f"Search results (distances): {distances}")
id_to_text_map = {i: f"Document ID {i}" for i in range(index.ntotal)}
print("\nCorresponding Document IDs for the search results:")
for idx in indices[0]:
    doc_id = id_to_text_map.get(idx, "Unknown Document ID")
    print(f"Index {idx} -> Document ID: {doc_id}")


Retrieved vectors: 
[[-0.00677118  0.02276473 -0.00233496 ...  0.02569435  0.02762516
  -0.01567653]
 [ 0.03025599  0.05168479 -0.02412735 ...  0.0234941   0.00523008
  -0.05033495]
 [-0.017729    0.02841178 -0.03170023 ...  0.02352753  0.01174562
  -0.06735812]
 ...
 [-0.03484033  0.02584333 -0.02750481 ...  0.03353063 -0.00916356
  -0.07829966]
 [-0.02960305  0.01953099 -0.00545851 ...  0.00705053 -0.01074381
  -0.04163727]
 [-0.07328457  0.00565345 -0.01656812 ... -0.01026038  0.01278077
  -0.02600808]]
Search results (distances): [[341.71185 341.82825 341.97366 342.00555 342.0192 ]]

Corresponding Document IDs for the search results:
Index 409 -> Document ID: Document ID 409
Index 1092 -> Document ID: Document ID 1092
Index 64 -> Document ID: Document ID 64
Index 480 -> Document ID: Document ID 480
Index 2314 -> Document ID: Document ID 2314
